In [46]:
import bs4
from langchain_text_splitters import HTMLHeaderTextSplitter
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths = ("https://namu.wiki/w/%EB%AA%85%EC%98%88%ED%9B%88%EC%9E%A5(%EC%9D%B8%ED%84%B0%EB%84%B7%20%EB%B0%A9%EC%86%A1%EC%9D%B8)",),
)
docs = loader.load()

docs

In [51]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader('namuwiki.txt')
docs = loader.load()
docs

[Document(metadata={'source': 'namuwiki.txt'}, page_content="닉네임인 명예훈장을 줄인 명훈[9] 혹은 '띵훈'으로 잘 알려져있다. 버블파이터와 오버워치의 프로게이머로 활동했다. 닉네임의 유래는 버블파이터에서 게임 결과창에 나오는 훈장 시스템을 보고 만들었다고 한다.\n\n버블파이터 내에서 오랜기간 1위를 유지 했었던 유명한 랭커 출신으로, 아프리카TV 최초로 버블파이터 베스트BJ에 도달했다.[10] 2015년 10월쯤 트레져헌터에 입사하고, 프로게이머 BJ 칭호를 받았다. 현재는 카론 크리에이티브에 소속되어있다.\n\n오버워치에서는 겐지, 리퍼 플레이로 대한민국 서버 1위와 전세계 랭킹 6위[11]를 달성하는 엄청난 저력을 보이며 천상계 랭커 자리를 유지했으며, 대한민국 3대 겐지[명단]라는 이명으로 불리면서 오버워치를 플레이하는 유저들이라면 익히 한번씩은 들어봤을만한 인물들 중 한명이다.\n\n지인들과 결성한 'CONBOX T6'로 오버워치 프로게이머로 데뷔했으며, Lunatic-Hai에 재입단을 했지만 스트리밍에 본격적으로 집중하겠다는 자신의 굳은 의지로 팀을 나오게 되며 현재는 인터넷 방송인으로써 방송에서 다양한 게임을 플레이하고 있다.[13]\n\n아프리카TV에서 방송을 하고 있었으나 갑질사태를 기점으로 2016년 10월 26일 트위치로 방송국을 이전하였고, 트위치가 트위치 대한민국 사업 철수 사건으로 철수를 하자, 유튜브와 치지직, 아프리카TV 3개의 플랫폼에서 동시송출을 했다. 2024년 8월 13일부터는 치지직 파트너 계약 및 단독 송출을 하는 중이다.\n")]

In [52]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
import os
import mykey
from langchain_text_splitters import CharacterTextSplitter

os.environ['OPENAI_API_KEY'] = mykey.openai()
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0.1,
)

text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
embeddings = OpenAIEmbeddings()
split_docs = text_splitter.split_documents(docs)

In [53]:
db = FAISS.from_documents(split_docs, embeddings)

In [54]:
retreieved = db.as_retriever()

In [61]:
docs = retreieved.invoke('오버워치', k=1)

In [62]:
for doc in docs:
    print(doc.page_content)
    print('-'*20)

오버워치에서는 겐지, 리퍼 플레이로 대한민국 서버 1위와 전세계 랭킹 6위[11]를 달성하는 엄청난 저력을 보이며 천상계 랭커 자리를 유지했으며, 대한민국 3대 겐지[명단]라는 이명으로 불리면서 오버워치를 플레이하는 유저들이라면 익히 한번씩은 들어봤을만한 인물들 중 한명이다.

지인들과 결성한 'CONBOX T6'로 오버워치 프로게이머로 데뷔했으며, Lunatic-Hai에 재입단을 했지만 스트리밍에 본격적으로 집중하겠다는 자신의 굳은 의지로 팀을 나오게 되며 현재는 인터넷 방송인으로써 방송에서 다양한 게임을 플레이하고 있다.[13]
--------------------


In [64]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)

In [65]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"context": retreieved, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [68]:
chain.invoke("명예훈장 오버워치 경력")

'명예훈장은 오버워치에서 겐지, 리퍼 플레이로 대한민국 서버 1위와 전세계 랭킹 6위를 달성한 프로게이머입니다.'

In [69]:
chain.invoke("명예훈장 겐지에 대해 설명해줘")

'명예훈장은 오버워치에서 겐지를 플레이하는 대한민국의 프로게이머이며, 서버 1위와 전세계 랭킹 6위를 달성한 유명한 랭커입니다. CONBOX T6로 데뷔하여 현재는 인터넷 방송인으로 활동하고 있습니다.'

In [70]:
model.invoke("명예훈장 겐지에 대해 설명해줘")

AIMessage(content="명예훈장 겐지는 일본의 전통적인 귀족 제도인 '겐지'를 받은 사람들에게 수여되는 명예 훈장입니다. 겐지는 고대 일본에서 귀족층을 나타내는 신분을 뜻하며, 이를 받은 사람들은 일본 사회에서 높은 지위와 존경을 받는다고 알려져 있습니다. 이 명예훈장은 일본 정부가 공식적으로 수여하며, 일본 사회에서의 영향력과 명예를 상징하는 중요한 상징입니다.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 191, 'prompt_tokens': 25, 'total_tokens': 216, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d8718b1d-f706-41d1-b63b-66f484424892-0', usage_metadata={'input_tokens': 25, 'output_tokens': 191, 'total_tokens': 216, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})